In [1]:
from google.colab import files
uploaded = files.upload()
import pandas as pd


Saving ner_dataset.csv to ner_dataset.csv


In [2]:
import pandas as pd
import numpy as np
import spacy
import requests
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, TimeDistributed, Dense


In [3]:
ner_data=pd.read_csv("ner_dataset.csv", encoding='unicode_escape',on_bad_lines='skip')
ner_data

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
1048570,NaN,they,PRP,O
1048571,NaN,responded,VBD,O
1048572,NaN,to,TO,O
1048573,NaN,the,DT,O


In [4]:
def process_ner_data(data):
    data['Sentence #']= data['Sentence #'].fillna(method='ffill')
    grouped_df=data.groupby('Sentence #')
    sentense=grouped_df['Word'].apply(list).tolist()
    tags =grouped_df['Tag'].apply(list).tolist()
    return sentense,tags
sentences,tags =  process_ner_data(ner_data)
print(sentences[2])
print(tags[2][2])


<ipython-input-4-3446745710>:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['Sentence #']= data['Sentence #'].fillna(method='ffill')


['Helicopter', 'gunships', 'Saturday', 'pounded', 'militant', 'hideouts', 'in', 'the', 'Orakzai', 'tribal', 'region', ',', 'where', 'many', 'Taliban', 'militants', 'are', 'believed', 'to', 'have', 'fled', 'to', 'avoid', 'an', 'earlier', 'military', 'offensive', 'in', 'nearby', 'South', 'Waziristan', '.']
B-tim


In [5]:
# Drop rows with missing words or tags (just to be safe)

cleaned_sentences = []
cleaned_tags = []

for sent, tag_seq in zip(sentences, tags):
    if isinstance(sent, list) and isinstance(tag_seq, list):
        clean_words = []
        clean_tags = []
        for word, tag in zip(sent, tag_seq):
            if isinstance(word, str) and isinstance(tag, str):
                clean_words.append(word)
                clean_tags.append(tag)
        if word and clean_tags:
            cleaned_sentences.append(clean_words)
            cleaned_tags.append(clean_tags)


word_set = set()
tag_set = set()
for sentence in cleaned_sentences:
    word_set.update(sentence)
for tag_seq in cleaned_tags:
    tag_set.update(tag_seq)

word_list = sorted(list(word_set))
tag_list = sorted(list(tag_set))

word_to_index = {word: idx + 2 for idx, word in enumerate(word_list)}
word_to_index["PAD"] = 0
word_to_index["UNK"] = 1

tag_to_index = {tag: idx for idx, tag in enumerate(tag_list)}
index_to_tag = {idx: tag for tag, idx in tag_to_index.items()}

tag_to_index

{'B-art': 0,
 'B-eve': 1,
 'B-geo': 2,
 'B-gpe': 3,
 'B-nat': 4,
 'B-org': 5,
 'B-per': 6,
 'B-tim': 7,
 'I-art': 8,
 'I-eve': 9,
 'I-geo': 10,
 'I-gpe': 11,
 'I-nat': 12,
 'I-org': 13,
 'I-per': 14,
 'I-tim': 15,
 'O': 16}

In [6]:
max_len = max(len(s) for s in cleaned_sentences)
x=[[word_to_index.get(word,word_to_index["UNK"])for word in sent]for sent in cleaned_sentences]
x=pad_sequences(maxlen=max_len,sequences=x,padding="post",value=word_to_index["PAD"])
y=[[tag_to_index[tag] for tag in tag_seq] for tag_seq in cleaned_tags ]
y=pad_sequences(maxlen=max_len,sequences=y,padding="post", value=tag_to_index["O"])
y = [to_categorical(i, num_classes=len(tag_to_index)) for i in y]
X_train,X_val, y_train, y_val=train_test_split(x,y,test_size=0.1,random_state=42)

In [14]:
#Building a model to train the data.
def get_Bilstm_model():
  vocab_size = max(word_to_index.values()) + 1
  model=Sequential()
  model.add(Embedding(input_dim=vocab_size,
                               output_dim=128,
                               input_length=max_len,
                               mask_zero=False))
  model.add(Bidirectional(LSTM(units=64,return_sequences=True,dropout=0.2,recurrent_dropout=0.2)))
  model.add(LSTM(units=64,return_sequences=True,dropout=0.5,recurrent_dropout=0.5))
  model.add(TimeDistributed(Dense(len(tag_to_index),activation="softmax")))
  model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
  model.summary()
  return model
model=get_Bilstm_model()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
def train_model(X_train, y_train, X_val, y_val, model):
    history = model.fit(
        x=np.array(X_train),
        y=np.array(y_train),
        batch_size=32,
        epochs=5,
        validation_data=(np.array(X_val), np.array(y_val)),
        verbose=1
    )

    loss, accuracy = model.evaluate(np.array(X_val), np.array(y_val), verbose=1)
    return history, loss, accuracy


In [13]:
# Model Metrics
history, loss, accuracy = train_model(X_train, y_train, X_val, y_val, model)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")


Epoch 1/5
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 728s 540ms/step - accuracy: 0.9761 - loss: 0.0802 - val_accuracy: 0.9912 - val_loss: 0.0322
Epoch 2/5
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 739s 538ms/step - accuracy: 0.9924 - loss: 0.0280 - val_accuracy: 0.9926 - val_loss: 0.0256
Epoch 3/5
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 709s 526ms/step - accuracy: 0.9943 - loss: 0.0199 - val_accuracy: 0.9930 - val_loss: 0.0239
Epoch 4/5
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 708s 525ms/step - accuracy: 0.9951 - loss: 0.0167 - val_accuracy: 0.9932 - val_loss: 0.0234
Epoch 5/5
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 757s 536ms/step - accuracy: 0.9956 - loss: 0.0145 - val_accuracy: 0.9934 - val_loss: 0.0236
150/150 ━━━━━━━━━━━━━━━━━━━━ 13s 88ms/step - accuracy: 0.9935 - loss: 0.0233
Validation Loss: 0.0236
Validation Accuracy: 0.9934


In [17]:
#Testing model
def predict_entities(text, model, word_to_index, index_to_tag, max_len=max_len):
    tokens = text.split()
    token_indices = [word_to_index.get(token, word_to_index["UNK"]) for token in tokens]
    input_data = pad_sequences([token_indices], maxlen=max_len, padding='post', value=word_to_index["PAD"])

    pred_probs = model.predict(input_data)
    pred_indices = np.argmax(pred_probs, axis=-1)[0][:len(tokens)]
    pred_tags = [index_to_tag.get(idx, "O") for idx in pred_indices]

    return list(zip(tokens, pred_tags))


In [22]:
test_sentence = "OpenAI is based in San Francisco."
entities = predict_entities(test_sentence, model, word_to_index, index_to_tag)

for word, tag in entities:
    print(f"{word} --> {tag}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
OpenAI --> I-geo
is --> B-tim
based --> I-per
in --> I-per
San --> I-per
Francisco. --> I-per


In [27]:
import spacy
from spacy import displacy
nlp_spacy = spacy.load("en_core_web_sm")
doc = nlp_spacy("OpenAI is based in San Francisco.")
print([(ent.text, ent.label_) for ent in doc.ents])
displacy.render(doc, style="ent", jupyter=True)

[('OpenAI', 'GPE'), ('San Francisco', 'GPE')]
